# Classification, Segmentation and Detection of tumors in mammography medical images (mejorar)

## Authors:
- **Sebastián Kay Conde Lorenzo (correo aquí)**
- **Jaime Capdepon Fraile (correo aquí)**
- **Joaquín Negrete Saab (correo aquí)**
- **Yang Liu (correo aquí)**
- **Christian Most Tazon (correo aquí)**
  
## Summary:
aquí una descripción

---



## Imports

In [2]:
# Third library imports
import typing
import numpy as np
import cv2 as cv
import torch
import torch.nn as nn

# Custom imports
import dir_manager
import viewer
import models

## Data download

Aquí deberíamos poner que los datos se descarguen de una API o bien se saquen de una BBDD (a discutir)

## Directory tree creation

In [ ]:
# Crear objeto dir_manager.DirectoryManager()

## Dataset visualization

In [ ]:
# Crear objeto viewer.Visualizer()